# Vanilla AutoEncoder Example with CIFAR10
### Emre Can Acikgoz

# Imports

In [1]:
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


# Configurations

In [2]:
EPOCHS = 5
BATCH_SIZE = 32
LR = 1e-3
WEIGHT_DECAY = 0

# Dataloader and Pre-processing

In [3]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.4914, 0.4822, 0.4466), (0.247,0.243, 0.261))])

train = datasets.CIFAR10(root="./data", train=True,  download=True, transform=transform)
test  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test,  batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


# Model

### Encoder

In [4]:
class encoder(nn.Module):
    def __init__(self, in_channel=3, latent_dim=8):
        super(encoder, self).__init__()
        self.down_sample1 = nn.Conv2d(in_channels=in_channel, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.down_sample2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.down_sample3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.relu3 = nn.ReLU(inplace=True)
        self.down_sample4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU(inplace=True)
        self.down_sample5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.relu5 = nn.ReLU(inplace=True)
        self.fc = nn.Linear(64*4*4, latent_dim)
        
    def forward(self, x):
        #print(f"x: {x.shape}")
        x = self.down_sample1(x)
        #print(f"down_sample1: {x.shape}")
        x = self.relu1(x)
        
        x = self.down_sample2(x)
        #print(f"down_sample2: {x.shape}")
        x = self.relu2(x)
        
        x = self.down_sample3(x)
        #print(f"down_sample3: {x.shape}")
        x = self.relu3(x)
        
        x = self.down_sample4(x)
        #print(f"down_sample4: {x.shape}")
        x = self.relu4(x)
        
        x = self.down_sample5(x)
        #print(f"down_sample5: {x.shape}")
        out = self.relu5(x)
        
        return out
        

### Decoder

In [5]:
class decoder(nn.Module):
    def __init__(self, in_channels=8):
        super(decoder, self).__init__()
        self.up_sample1 = nn.ConvTranspose2d(in_channels=64, out_channels= 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.up_sample2 = nn.ConvTranspose2d(in_channels=32, out_channels= 32, kernel_size=3, stride=1, padding=1, output_padding=0)
        self.relu2 = nn.ReLU(inplace=True)
        self.up_sample3 = nn.ConvTranspose2d(in_channels=32, out_channels= 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.relu3 = nn.ReLU(inplace=True)
        self.up_sample4 = nn.ConvTranspose2d(in_channels=16, out_channels= 16, kernel_size=3, stride=1, padding=1, output_padding=0)
        self.relu4 = nn.ReLU(inplace=True)
        self.up_sample5 = nn.ConvTranspose2d(in_channels=16, out_channels= 3,  kernel_size=3, stride=2, padding=1, output_padding=1)
        
    def forward(self, x):
        x = self.up_sample1(x)
        #print(f"up_sample1: {x.shape}")
        x = self.relu1(x)
        
        x = self.up_sample2(x)
        #print(f"up_sample2: {x.shape}")
        x = self.relu2(x)
        
        x = self.up_sample3(x)
        #print(f"up_sample3: {x.shape}")
        x = self.relu3(x)
        
        x = self.up_sample4(x)
        #print(f"up_sample4: {x.shape}")
        x = self.relu4(x)
        
        out = self.up_sample5(x)
        #print(f"up_sample5: {x.shape}")
        
        return out

### AutoEncoder

In [6]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = encoder()
        self.fc1 = nn.Linear(64*4*4, 8)
        self.fc2 = nn.Linear(8, 64*4*4)
        self.relu = nn.ReLU(inplace=True)
        self.decoder = decoder()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.encoder(x)
        #print(f"encoder: {x.shape}")
        x = x.view(x.shape[0], -1)
        #print(f"x: {x.shape}")
        x = self.fc1(x)
        #print(f"fc1: {x.shape}")
        x = self.fc2(x)
        #print(f"fc2: {x.shape}")
        x = x.reshape(x.shape[0], -1, 4, 4)
        #print(f"x: {x.shape}")
        x = self.decoder(x)
        #print(f"decoder: {x.shape}")
        out = self.sigmoid(x)
        
        return out
        
model = autoencoder().to(device)

# Loss and Optimizer

In [7]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,20], gamma=0.1)

# Training

In [ ]:
for epoch in range(EPOCHS):
    running_train_loss = 0
    losses = []
    outputs = []
    for image, target in train_loader:
        image, _ = image.to(device), target.to(device)
        
        output = model(image)
        loss = criterion(output, image)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
        outputs.append((epoch+1, image, output))
    loss = running_train_loss / len(train_loader)
    losses.append(loss)
    print(f"Epoch [{epoch+1}/{EPOCHS}], loss:{loss:.4f}")

Epoch [1/5], loss:0.6809
Epoch [2/5], loss:0.6734
